In [1]:
import pandas as pd

In [2]:
# sorted_events = pd.read_csv('data/events.csv')
sorted_events = pd.read_csv('/Users/rishabhshah/Desktop/AIPI531/HW3/Kaggle/data/sorted_events.csv')

item_properties_part1 = pd.read_csv('data/item_properties_part1.csv')
item_properties_part2 = pd.read_csv('data/item_properties_part2.csv')
item_properties = pd.concat([item_properties_part1, item_properties_part2], ignore_index=True)

In [6]:
# Remove the 'timestamp' column from the item_properties dataframe and drop any duplicate rows
item_properties_ = item_properties.drop(['timestamp'], axis=1).drop_duplicates()

# Get a list of unique item IDs from the sorted_events dataframe
item_list = sorted_events['item_id'].unique()
item_list.sort()

# Filter the item_properties_ dataframe to only include items that are in the item_list and drop any duplicate rows
item_properties_ = item_properties_[item_properties_['itemid'].isin(item_list)].drop_duplicates()

# droppig rows from item_properties_ where the 'property' is 'available' and the 'value' is '0'. This effectively removes any items that are not available.
item_properties_ = item_properties_[~((item_properties_['property'] == "available") &
                        (item_properties_['value'] == '0'))]

# Create a new column in item_prop called 'prop_val' that is a combination of the 'property' and 'value' columns. This is done by stripping any leading or trailing whitespace from the 'property' and 'value' columns and then concatenating them together.
item_properties_['prop_val'] = item_properties_['property'].str.strip() + item_properties_['value'].str.strip()

In [7]:
# # select top 400 properties
properties = item_properties_['prop_val'].value_counts().head(400).index.tolist()

# # Initialize list
item_prop_encoded = []

# # Go through each item
for item in item_list.tolist():
    item_set = set(item_properties_[item_properties_["itemid"] == item]['prop_val'].unique())
    new_row = [1 if prop in item_set else 0 for prop in properties]
    item_prop_encoded.append(new_row)

In [ ]:
item_list.shape, item_prop.shape

((70852,), (1882863, 4))

In [ ]:
# convert to df
item_prop_encoded_df = pd.DataFrame(item_prop_encoded)

# export to CSVs
item_prop_encoded_df.to_csv('kaggle/data/item_prop_ohe.csv', index=False)